<a href="https://colab.research.google.com/github/1900690/yolo-traking/blob/main/yolo_traking_panorama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#果実トラッキング

※学習前のＡＩで追跡を行う。追跡できる品目はリンゴ、オレンジなど代表的なもののみ

In [1]:
#@title 分析する動画をアップロードまたはサンプル動画(ユズ)で分析

#事前準備
!pip -q install ultralytics

from google.colab import files
import os
サンプル動画を使用する = False # @param {type:"boolean"}

if サンプル動画を使用する == True:
  if os.path.exists('/content/yuzu.mp4'):
     os.remove('/content/yuzu.mp4')
  !wget -q  https://github.com/1900690/yolo-traking/releases/download/sample/yuzu.mp4

else:
  uploaded_file = files.upload()
  uploaded_file_name = next(iter(uploaded_file))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 kB 14.1 MB/s eta 0:00:00


Saving 60fpsリンゴ.mp4 to 60fpsリンゴ (1).mp4


<p align="center">
  <img src="https://github.com/1900690/yolo-traking/blob/main/images/yuzu_before.jpg?raw=1" width=80%>
  <br>
  ユズのサンプル動画
</p>

In [2]:
#@markdown 分析して動画をダウンロード
#因数は以下を参照https://docs.ultralytics.com/modes/predict/#inference-arguments
from ultralytics import YOLO
import shutil
import os
from google.colab import files

if os.path.exists('/content/runs'):
    shutil.rmtree('/content/runs')

# Load a model
model = YOLO("yolo11x.pt")

if サンプル動画を使用する == True:
  results = model.track(source="/content/yuzu.mp4",stream=False,save=True)
  files.download("/content/runs/detect/track/yuzu.avi")
else:
  results = model.track(source="/content/"+uploaded_file_name,stream=False,save=True,classes=47,save_txt=True,show_conf=False)
  files.download("/content/runs/detect/track/"+uploaded_file_name[:-4]+".avi")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 109M/109M [00:00<00:00, 187MB/s] 


requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.4s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1115) /content/60fpsリンゴ (1).mp4: 384x640 22 apples, 3415.9ms
video 1/1 (frame 2/1115) /content/60fpsリンゴ (1).mp4: 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<p align="center">
  <img src="https://github.com/1900690/yolo-traking/blob/main/images/yuzu_after.jpg?raw=1" width=80%>
  <br>
  果実数を検出後のユズのサンプル動画
</p>

In [3]:
import cv2

from ultralytics import solutions


def count_objects_in_region(video_path, output_video_path, model_path):
    """Count objects in a specific region within a video."""
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    region_points = [(20, 400), (1080, 404), (1080, 360), (20, 360)]
    counter = solutions.ObjectCounter(show=True, region=region_points, model=model_path)

    while cap.isOpened():
        success, im0 = cap.read()
        if not success:
            print("Video frame is empty or video processing has been successfully completed.")
            break
        im0 = counter.count(im0)
        video_writer.write(im0)

    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()


count_objects_in_region("/content/60fpsリンゴ.mp4", "/content/60fpsリンゴ.avi", "yolo11n.pt")

Ultralytics Solutions: ✅ {'region': [(20, 400), (1080, 404), (1080, 360), (20, 360)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'show': True, 'model': 'yolo11n.pt'}


100%|██████████| 5.35M/5.35M [00:00<00:00, 19.9MB/s]


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


0: 384x640 24 apples, 1 orange, 171.1ms
Speed: 2.9ms preprocess, 171.1ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 apples, 1 orange, 137.2ms
Speed: 4.9ms preprocess, 137.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 apples, 1 orange, 142.8ms
Speed: 2.9ms preprocess, 142.8ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 apples, 1 orange, 137.6ms
Speed: 2.3ms preprocess, 137.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 apples, 1 orange, 148.2ms
Speed: 2.8ms preprocess, 148.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 apples, 1 orange, 163.0ms
Speed: 2.7ms preprocess, 163.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 apples, 1 orange, 137.3ms
Speed: 3.6ms preprocess, 137.3ms inference, 1.4ms p

In [ ]:
import cv2

from ultralytics import solutions

cap = cv2.VideoCapture("/content/60fpsリンゴ.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define region points
region_points = [(20, 400), (1080, 400)]  # For line counting
#region_points = [(20, 400), (1080, 404), (1080, 360), (20, 360)]  # For rectangle region counting
# region_points = [(20, 400), (1080, 404), (1080, 360), (20, 360), (20, 400)]  # For polygon region counting

# Video writer
video_writer = cv2.VideoWriter("/content/60fpsリンゴ.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init Object Counter
counter = solutions.ObjectCounter(
    #show=True,  # Display the output
    region=region_points,  # Pass region points
    model="yolo11x.pt",  # model="yolo11n-obb.pt" for object counting using YOLO11 OBB model.
    classes=[47],  # If you want to count specific classes i.e person and car with COCO pretrained model.
    # show_in=True,  # Display in counts
    # show_out=True,  # Display out counts
    # line_width=2,  # Adjust the line width for bounding boxes and text display
)

# Process video
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    im0 = counter.count(im0)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()